<a href="https://colab.research.google.com/github/dish-hash/Job-market/blob/main/bde_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall pyspark

Found existing installation: pyspark 3.5.3
Uninstalling pyspark-3.5.3:
  Would remove:
    /usr/local/bin/beeline
    /usr/local/bin/beeline.cmd
    /usr/local/bin/docker-image-tool.sh
    /usr/local/bin/find-spark-home
    /usr/local/bin/find-spark-home.cmd
    /usr/local/bin/find_spark_home.py
    /usr/local/bin/load-spark-env.cmd
    /usr/local/bin/load-spark-env.sh
    /usr/local/bin/pyspark
    /usr/local/bin/pyspark.cmd
    /usr/local/bin/pyspark2.cmd
    /usr/local/bin/run-example
    /usr/local/bin/run-example.cmd
    /usr/local/bin/spark-class
    /usr/local/bin/spark-class.cmd
    /usr/local/bin/spark-class2.cmd
    /usr/local/bin/spark-connect-shell
    /usr/local/bin/spark-shell
    /usr/local/bin/spark-shell.cmd
    /usr/local/bin/spark-shell2.cmd
    /usr/local/bin/spark-sql
    /usr/local/bin/spark-sql.cmd
    /usr/local/bin/spark-sql2.cmd
    /usr/local/bin/spark-submit
    /usr/local/bin/spark-submit.cmd
    /usr/local/bin/spark-submit2.cmd
    /usr/local/bin/sparkR
  

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=632fdff45334902602547cfa0e5682f6715e135541c8cb815f08a65817db0fb3
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [ ]:
from google.colab import files
uploaded = files.upload()
import pyarrow.parquet as pq
import pyarrow.fs as fs
hdfs = fs.HadoopFileSystem('hdfs://namenode_host:9000')
hdfs_path = '/Users/Jin boy/Documents/job market dataset/Naukri_Data_Scientist_and_Data_Analytics_Jobs_Data.parquet'

# Read the Parquet dataset from HDFS
dataset = pq.ParquetDataset(hdfs_path, filesystem=hdfs)

Saving Naukri_Data_Scientist_and_Data_Analytics_Jobs_Data.csv to Naukri_Data_Scientist_and_Data_Analytics_Jobs_Data.csv


In [ ]:
# Install PySpark
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz
!tar xf spark-3.4.0-bin-hadoop3.tgz
!pip install -q findspark


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:3 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:4 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,452 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,696 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd

In [ ]:
# Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.0-bin-hadoop3"

# Initialize PySpark
import findspark
findspark.init()

# Import and start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

# Check if Spark is running
spark


In [ ]:
# Upload CSV file
from google.colab import files
uploaded = files.upload()

# Define the file path after uploading
file_path = "/content/Naukri_Data_Scientist_and_Data_Analytics_Jobs_Data.csv"

# Read the CSV file into a PySpark DataFrame
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Show the first few rows
df.show(5)


Saving Naukri_Data_Scientist_and_Data_Analytics_Jobs_Data.csv to Naukri_Data_Scientist_and_Data_Analytics_Jobs_Data (1).csv
+--------------------+--------------------+-------------------+-------------+--------------------+--------------------+
|          Job Titles|       Company Names|Experience Required|      Package|           Locations|              Skills|
+--------------------+--------------------+-------------------+-------------+--------------------+--------------------+
|Manager - Digital...|                Resy|            4-8 Yrs|Not disclosed|    Gurgaon/Gurugram|Product managemen...|
|Data Science Doma...|            Coursera|           7-11 Yrs|Not disclosed|Kolkata, Mumbai, ...|Computer scienceC...|
|GN - Strategy - M...|           Accenture|            1-3 Yrs|Not disclosed|Mumbai, Hyderabad...|Change management...|
|Data Science Manager|Foreign IT Consul...|            2-6 Yrs|Not disclosed|    Gurgaon/Gurugram|Operations resear...|
|Data Science Manager|Foreign IT Con

In [ ]:
from pyspark.sql import Window
from pyspark.sql.functions import last, col

# Define a window specification to order by 'Job Titles' (or another suitable column)
window_spec = Window.orderBy("Job Titles").rowsBetween(Window.unboundedPreceding, 0)

# Forward fill the 'Experience Required' column
df = df.withColumn("Experience Required", last(col("Experience Required"), ignorenulls=True).over(window_spec))

# Forward fill the 'Company Names' column
df = df.withColumn("Company Names", last(col("Company Names"), ignorenulls=True).over(window_spec))

# Show the DataFrame to verify forward filling
df.show(5)


+----------+-------------+-------------------+-------------+--------------------+--------------------+
|Job Titles|Company Names|Experience Required|      Package|           Locations|              Skills|
+----------+-------------+-------------------+-------------+--------------------+--------------------+
|      null|         null|           6-10 Yrs|Not disclosed|Gurgaon/ Gurugram...|Advanced Statisti...|
|      null|         null|           8-10 Yrs|Not disclosed|Hyderabad/Secunde...|Statistical progr...|
|      null|         null|          12-15 Yrs|Not disclosed|Hybrid - Noida, U...|NLPData SciencePr...|
|      null|         null|            7-9 Yrs|Not disclosed|             Chennai|SQL queriesData a...|
|      null|         null|            3-7 Yrs|10-20 Lacs PA|Hybrid - Gurgaon/...|SQLBusiness Intel...|
+----------+-------------+-------------------+-------------+--------------------+--------------------+
only showing top 5 rows



In [ ]:
# Remove duplicate rows
df = df.dropDuplicates()

# Show the count of rows after removing duplicates
print("Number of rows after removing duplicates:", df.count())


Number of rows after removing duplicates: 19646


In [ ]:
indexer = StringIndexer(inputCol="Job Titles", outputCol="New_Job_Title_Index", handleInvalid="skip")
df = indexer.fit(df).transform(df)


In [ ]:
df = df.drop("Job_Title_Index")
indexer = StringIndexer(inputCol="Job Titles", outputCol="Job_Title_Index", handleInvalid="skip")
df = indexer.fit(df).transform(df)


In [ ]:
# Drop the existing column if it exists
if "Job_Title_Index" in df.columns:
    df = df.drop("Job_Title_Index")

# Create and apply the StringIndexer
indexer = StringIndexer(inputCol="Job Titles", outputCol="Job_Title_Index", handleInvalid="skip")
df = indexer.fit(df).transform(df)


In [ ]:
# Fill nulls in 'Experience Required' with a default value, e.g., "0 years"
df = df.fillna({'Experience Required': '0 years'})


In [ ]:
# Fill nulls in the 'Package' column
df = df.fillna({'Package': 'Not Specified'})


In [ ]:
# Fill nulls in multiple columns
df = df.fillna({
    'Experience Required': '0 years',
    'Package': 'Not Specified',
    'Locations': 'Unknown'
})


In [ ]:
from pyspark.sql.functions import count, when, col


In [ ]:
from pyspark.sql.functions import count, when, col

# Count null values in each column
null_counts = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])
null_counts.show()


+----------+-------------+-------------------+-------+---------+------+------------------+-------------------+---------------+
|Job Titles|Company Names|Experience Required|Package|Locations|Skills|Company_Name_Index|New_Job_Title_Index|Job_Title_Index|
+----------+-------------+-------------------+-------+---------+------+------------------+-------------------+---------------+
|         0|            0|                  0|      0|        0|     0|                 0|                  0|              0|
+----------+-------------+-------------------+-------+---------+------+------------------+-------------------+---------------+



In [ ]:
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("Example").getOrCreate()

# Sample DataFrame creation (replace this with your actual DataFrame)
data = [("A",), ("B",), ("C",), ("A",), ("B",)]
df = spark.createDataFrame(data, ["Package"])

# Step 1: Index the 'Package' column
indexer = StringIndexer(inputCol="Package", outputCol="Package_Index")
df = indexer.fit(df).transform(df)

# Step 2: Assemble the indexed column into a vector
assembler = VectorAssembler(inputCols=["Package_Index"], outputCol="Package_Feature")
df = assembler.transform(df)

# Show the transformed DataFrame
df.show()


+-------+-------------+---------------+
|Package|Package_Index|Package_Feature|
+-------+-------------+---------------+
|      A|          0.0|          [0.0]|
|      B|          1.0|          [1.0]|
|      C|          2.0|          [2.0]|
|      A|          0.0|          [0.0]|
|      B|          1.0|          [1.0]|
+-------+-------------+---------------+



In [ ]:
# Split the data into training and test sets (80% training, 20% testing)
train_df, test_df = df.randomSplit([0.8, 0.2], seed=42)

print("Training Dataset Count:", train_df.count())
print("Test Dataset Count:", test_df.count())


Training Dataset Count: 4
Test Dataset Count: 1


In [ ]:
df.printSchema()


root
 |-- Package: string (nullable = true)
 |-- Package_Index: double (nullable = false)
 |-- Package_Feature: vector (nullable = true)



In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType
from google.colab import files

# Create a Spark session
spark = SparkSession.builder.appName("MyApp").getOrCreate()

# Your existing DataFrame operations
# df = ... (your DataFrame after preprocessing)

# Check the schema of the DataFrame
df.printSchema()

# Define a UDF to convert array to string
array_to_string_udf = udf(lambda arr: ",".join(map(str, arr)) if arr is not None else None, StringType())

# Apply the UDF to convert the array column to string
df = df.withColumn("Package_Feature", array_to_string_udf(col("Package_Feature")))

# Save the cleaned DataFrame to a new CSV file
output_file_path = "/content/cleaned_data"

# Remove previous directory if it exists (this won't cause an error if it doesn't exist)
!rm -r /content/cleaned_data

# Write the DataFrame to CSV
df.write.option("header", "true").csv(output_file_path)

# Confirm the files are saved
print(f"Cleaned DataFrame saved to {output_file_path}")

# Optional: Zip and download the CSV files
!zip -r cleaned_data.zip /content/cleaned_data
files.download('cleaned_data.zip')


root
 |-- Package: string (nullable = true)
 |-- Package_Index: double (nullable = false)
 |-- Package_Feature: array (nullable = true)
 |    |-- element: double (containsNull = true)

rm: cannot remove '/content/cleaned_data': No such file or directory
Cleaned DataFrame saved to /content/cleaned_data
  adding: content/cleaned_data/ (stored 0%)
  adding: content/cleaned_data/_SUCCESS (stored 0%)
  adding: content/cleaned_data/.part-00000-def9c334-1df9-49de-87f0-58520b3e1278-c000.csv.crc (stored 0%)
  adding: content/cleaned_data/part-00001-def9c334-1df9-49de-87f0-58520b3e1278-c000.csv (deflated 26%)
  adding: content/cleaned_data/.part-00001-def9c334-1df9-49de-87f0-58520b3e1278-c000.csv.crc (stored 0%)
  adding: content/cleaned_data/part-00000-def9c334-1df9-49de-87f0-58520b3e1278-c000.csv (deflated 26%)
  adding: content/cleaned_data/._SUCCESS.crc (stored 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>